## Initial setup

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
!pip3 install --upgrade google-cloud-aiplatform --user -q

Restart kernel

In [ ]:
# Fill appropriate values..
PROJECT_ID = ""
REGION = "us-central1"
BUCKET_NAME = ""
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

In [ ]:
aiplatform.init(project=PROJECT_ID,
                location=REGION,
                staging_bucket=BUCKET_URI)

## Create custom container image

Create repository in GCP Artifact Registry

In [ ]:
REPO_NAME='census-income'

!gcloud artifacts repositories create $REPO_NAME \
--repository-format=docker \
--location=$REGION \
--description="Docker repository"

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

In [ ]:
IMAGE_URI = (
             f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/xgboost_hptune:anurag"
            )

In [ ]:
!ls

Write a Dockerfile for your custom container

## Build image and push to Artifact Registry

In [ ]:
!docker build ./ -t $IMAGE_URI

In [ ]:
!docker push $IMAGE_URI

In [ ]:
IMAGE_URI

Check image on GCP console

## Define specs

Choose your hardware

In [ ]:
worker_pool_specs = [
                        {
                            "machine_spec": {
                                            "machine_type": "n1-standard-4",
                                            "accelerator_type": "NVIDIA_TESLA_T4",
                                            "accelerator_count": 1,
                                            },
                            "replica_count": 1,
                            "container_spec": {
                                                "image_uri": IMAGE_URI
                                              },
                        }
                    ]

Set the search space for your chosen hyper-parameters

In [ ]:
parameter_spec = {
                  "learning_rate": hpt.DoubleParameterSpec(min=0.001,  # float
                                                           max=0.1,  
                                                           scale="log"),
                  "max_depth": hpt.DiscreteParameterSpec(values=[5, 10, 15, 20],  # int
                                                         scale=None),
                  "scale_pos_weight": hpt.DiscreteParameterSpec(values=[1, 3, 5, 7],  # int
                                                                scale=None)
                 }

Set your model performance metric and the objective

In [ ]:
metric_spec={'roc_auc':'maximize'}

## Hyper-parameter Tuning job

Provide appropriate values to following arguments..

In [ ]:
display_name = "census-income-xgboost-hptune"

my_custom_job = aiplatform.CustomJob(
                                     display_name=display_name,
                                     worker_pool_specs=worker_pool_specs,
                                     staging_bucket=BUCKET_URI,
                                    )

In [ ]:
hp_job = aiplatform.HyperparameterTuningJob(
                                            display_name=display_name,
                                            custom_job=my_custom_job,
                                            metric_spec=metric_spec,
                                            parameter_spec=parameter_spec,
                                            max_trial_count=10,
                                            parallel_trial_count=3,
                                           )

hp_job.run()

Check output on GCP console